In [1]:
def t_cost(s,t): return ((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5)
def d_cost(s,t): return (1/alpha)*(((s[0]-t[0])**2+(s[1]-t[1])**2)**(0.5))
alpha=2

In [2]:
def t_pathcost(a,b,route):
    pathcost=0
    if a<b:
        for i in range(a,b):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
    if a>b:
        for i in range(a,len(route)):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
        for i in range(b):
            pathcost=pathcost+t_cost(V[route[i]],V[route[(i+1)%len(route)]])
    if a==b:
        pathcost=0
            
    return pathcost

In [3]:
import itertools
def two_opt(ps):
    n = len(ps)
    res = list(range(n))
    update = True
    while update:
        update = False
        for (i,j) in itertools.combinations(range(n),2):
            if ((t_cost(ps[res[i]],ps[res[i+1]])+t_cost(ps[res[j]],ps[res[(j+1)%n]]))>
                (t_cost(ps[res[i]],ps[res[j]])+t_cost(ps[res[i+1]],ps[res[(j+1)%n]]))):
                res[i+1:j+1] = res[j:i:-1]
                update = True
                
    return res

In [4]:
import heapq
def mst(ps):
    n = len(ps)
    #prim
    mst = {v:[] for v in range(n)} # MST の隣接リスト
    X = set()
    heap = []
    for v in range(1,n):
        heapq.heappush(heap,(t_cost(ps[0],ps[v]),0,v))
    while len(heap)>0:
        (d,u,v)=heapq.heappop(heap)
        if v not in X:
            X.add(v)
            mst[u].append(v)
            mst[v].append(u)
            for w in range(n):
                if w not in X: heapq.heappush(heap,(t_cost(ps[v],ps[w]),v,w))
    #dfs
    res = []
    stack = [0]
    visited = [False]*n
    while len(stack)>0:
        v = stack.pop()
        if visited[v]: continue
        visited[v]=True
        res.append(v)
        for u in mst[v]: stack.append(u)
    return res 

In [5]:
def dp(ps):
    length = {} # length[(u,S)]: u を始点とし S の点すべてを回る最小経路長、ディクショナリ
    route = {} # route[(u,S)]: 最小経路長を達成するためのルート、ディクショナリ
    v = ps[0]
    n = len(ps)
    for i in range(1,n+1):
        for a in itertools.combinations(range(n),i):
            S = frozenset(a)
            for j in S:
                u = ps[j]
                if i==1:
                    length[(j,S)] = t_cost(v,u)
                    route[(j,S)] = [j]
                else:
                    Sj = S-set([j]) # S から j を除いたもの
                    k=min(Sj,key=lambda k: length[(k,Sj)]+t_cost(ps[k],u))#Sjの中でlength[(k,Sj)]+dist(ps[k],u)が最小のものを返している
                    length[(j,S)] = length[(k,Sj)]+t_cost(ps[k],u)
                    route[(j,S)] = route[(k,Sj)]+[j]
    return route[(0,frozenset(range(n)))] 

In [26]:
import itertools
def merge(truck_nodes,drone_nodes,nd,alg,V,depot):
    D={}
    T={}
    t=[V[i] for i in truck_nodes]
    tpath=[truck_nodes[i] for i in alg(t)]
    for drone in drone_nodes:
        for (start,end) in itertools.permutations(tpath,2):
            drone_cost=d_cost(V[start],V[drone])+d_cost(V[drone],V[end])
            truck_cost=t_pathcost(tpath.index(start),tpath.index(end),tpath)
            T[start,drone,end]=max(drone_cost,truck_cost)
    #ドローンとトラックが一緒に移動する場合は考えない
    dc=[b for b in itertools.permutations(drone_nodes,nd)]#ドローンノードの訪れる順番
    tc=[tuple({depot})+a for a in itertools.combinations(set(tpath)-{depot},nd-1)]#トラックノードのうちどれをスタートノード、エンドノードとするか
    for comb in itertools.product(tc,dc):
        d=0
        for j in range(nd):
            d=d+T[comb[0][j],comb[1][j],comb[0][(j+1)%nd]]
        D[comb]=d,tpath
    return min(D.values()),min(D,key=D.get)

In [128]:
def merge_fast(truck_nodes,drone_nodes,nd,alg,V,depot):
    D={}
    T={}
    t=[V[i] for i in truck_nodes]
    tpath=[truck_nodes[i] for i in alg(t)]
    for drone in drone_nodes:
        for (start,end) in itertools.permutations(tpath,2):
            drone_cost=d_cost(V[start],V[drone])+d_cost(V[drone],V[end])
            truck_cost=t_pathcost(tpath.index(start),tpath.index(end),tpath)
            T[start,drone,end]=max(drone_cost,truck_cost)
    #ドローンとトラックが一緒に移動する場合は考えない
    d=[V[i] for i in drone_nodes]
    dpath=[drone_nodes[i] for i in alg(d)]
    dc=[tuple(dpath),tuple(reversed(dpath))]#ドローンノードの訪れる順番
    tc=[tuple({depot})+a for a in itertools.combinations(set(tpath)-{depot},nd-1)]#トラックノードのうちどれをスタートノード、エンドノードとするか
    for comb in itertools.product(tc,dc):
        d=0
        for j in range(nd):
            d=d+T[comb[0][j],comb[1][j],comb[0][(j+1)%nd]]
        D[comb]=d,tpath
    return min(D.values()),min(D,key=D.get)
            
    
            

In [129]:
def shiwake_mst(V,depot,dist):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_nodes=sorted(dist,key=dist.get)[-nd:]
        truck_nodes=list(set(range(n))-set(drone_nodes))   
        D[merge(truck_nodes,drone_nodes,nd,mst,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,mst,V,depot)[0]            
    res=min(D.values())[0]
    path=min(D,key=D.get)
    return res,path

In [151]:
def shiwake_mst_fast(V,depot,dist):
    n=len(V)
    ndmin=n//2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_nodes=sorted(dist,key=dist.get)[-nd:]
        truck_nodes=list(set(range(n))-set(drone_nodes))   
        D[merge_fast(truck_nodes,drone_nodes,nd,mst,V,depot)[1]]=merge_fast(truck_nodes,drone_nodes,nd,mst,V,depot)[0]            
    res=min(D.values())[0]
    path=min(D,key=D.get)
    return res,path

In [105]:
def shiwake_dp(V,depot,dist):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_nodes=sorted(dist,key=dist.get)[-nd:]
        truck_nodes=list(set(range(n))-set(drone_nodes))   
        D[merge(truck_nodes,drone_nodes,nd,dp,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,dp,V,depot)[0]            
    res=min(D.values())[0]
    path=min(D,key=D.get)
    return res,path

In [9]:
def MST_min(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return shiwake_mst(V,depot,dist)

def MST_sec(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return shiwake_mst(V,depot,dist)

def MST_sum(V,depot):
    n=len(V)
    dist={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return shiwake_mst(V,depot,dist)

def DP_min(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return shiwake_dp(V,depot,dist)

def DP_sec(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return shiwake_dp(V,depot,dist)

def DP_sum(V,depot):
    n=len(V)
    dist={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return shiwake_dp(V,depot,dist)

In [113]:
def MST_sum_fast(V,depot):
    n=len(V)
    dist={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return shiwake_mst_fast(V,depot,dist)


In [152]:
V=mondai_random(40)

In [153]:
%%time
MST_sum_fast(V,0)

Wall time: 280 ms


(4635.416097210977,
 ((0, 3, 4, 6, 7, 13, 14, 15, 17, 20, 23, 27, 29, 30, 32, 33, 35, 36, 37, 38),
  (34, 21, 24, 39, 10, 25, 26, 28, 18, 9, 31, 1, 2, 12, 16, 8, 11, 5, 19, 22)))

In [ ]:


def kouho_mst(V,depot,dist,m):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_subnodes=sorted(dist,key=dist.get)[-(nd+m):]
        for s in itertools.combinations(drone_subnodes,nd):
            drone_nodes=list(s)
            truck_nodes=list(set(range(n))-set(drone_nodes))

            D[merge(truck_nodes,drone_nodes,nd,mst,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,mst,V,depot)[0]

    res=min(D.values())[0]
    path=min(D,key=D.get)
    return res,path

In [11]:
def kouho_dp(V,depot,dist,m):
    n=len(V)
    ndmin=2
    ndmax=n//2
    D={}
    for nd in range(ndmin,ndmax+1):
        drone_subnodes=sorted(dist,key=dist.get)[-(nd+m):]
        for s in itertools.combinations(drone_subnodes,nd):
            drone_nodes=list(s)
            truck_nodes=list(set(range(n))-set(drone_nodes))

            D[merge(truck_nodes,drone_nodes,nd,dp,V,depot)[1]]=merge(truck_nodes,drone_nodes,nd,dp,V,depot)[0]

    res=min(D.values())[0]
    path=min(D,key=D.get)
    return res,path

In [12]:
def MST_min_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return kouho_mst(V,depot,dist,2)

def MST_sec_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return kouho_mst(V,depot,dist,2)

def MST_sum_kouho(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_mst(V,depot,wa,2)

def DP_min_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-2] for i in length}
    return kouho_dp(V,depot,dist,2)

def DP_sec_kouho(V,depot):
    n=len(V)
    length={i:[t_cost(V[i],V[j]) for j in range(n)] for i in set(range(n))-{depot}}
    dist={i:sorted(length[i])[-3] for i in length}
    return kouho_dp(V,depot,dist,2)

def DP_sum_kouho(V,depot):
    n=len(V)
    wa={i:sum([t_cost(V[i],V[j]) for j in range(n)]) for i in set(range(n))-{depot}}
    return kouho_dp(V,depot,wa,2)

In [13]:
def MST_dep(V,depot):
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}        
    return shiwake_mst(V,depot,dist)

def DP_dep(V,depot):
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}        
    return shiwake_dp(V,depot,dist)

def MST_dep_kouho(V,depot):
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}
    return kouho_mst(V,depot,dist,2)

def DP_dep_kouho(V,depot):
    dist={i:t_cost(V[depot],V[i]) for i in range(n)}
    return kouho_dp(V,depot,dist,2)

In [14]:
import random
def mondai_random(n):
    V=[(random.uniform(0,100),random.uniform(0,100)) for i in range(n)]
    return V

In [26]:
V=mondai_random(20)

In [27]:
%%time
MST_sum(V,0)

Wall time: 54.7 s


(1160.6762515384344,
 ((0, 1, 2, 3, 7, 8, 11, 14, 18, 19), (13, 4, 16, 6, 10, 9, 12, 15, 17, 5)))